In [ ]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [ ]:
# Import modules
import numpy as np # maths

from time import time# timer for debug

    # gather results obtained from the runs
import pickle # open .pkl files where python objects have been saved
from safe import safe # the empty class which is used to save the results
    
    # system commands
import os
    
    # interact with the C++ core and the Python core
import dimers as dim # C++ interface module
import KagomeFunctions as kf # "library" allowing to work on Kagome
import KagomeDrawing as kdraw # "library" allowing to plot Kagome
import Observables as obs #observables that can be used by the run script
import KagomeFT as kft #small functions to compute the FT
import AnalysisFunctions as af # functions to make the analysis code cleaner
    #plots using matplotlib
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams.update({'font.size': 14, 'pgf.texsystem':'pdflatex'})

In [ ]:
# Create a folder to save the pictures
foldername = '../../Analysis_PSI_EPFL/Runs_14-07-20_J1J2/'
results_foldername = 'J2CorrelationsScan/Results_20-07-20/'
filenamelist = [#'Crit_MediumKJ1J2_Corr_L4_J20.008_folder/backup',
                #'Crit_MediumKJ1J2_Corr_L4_J20.012_folder/backup',
                #'Crit_MediumKJ1J2_Corr_L4_J20.013_folder/backup',
                #'Crit_MediumKJ1J2_Corr_L4_J20.0135_folder/backup',
                #'Crit_MediumKJ1J2_Corr_L4_J20.014_folder/backup',
                #'Crit_MediumKJ1J2_Corr_L4_J20.015_folder/backup',
                #'Crit_MediumKJ1J2_Corr_L4_J20.017_folder/backup',
                'Crit_MediumKJ1J2_Corr_L4_J20.02_folder/backup',
                'MediumKJ1J2_Corr_L8_J20.03_folder/backup'
               ]

filenamelist = [#'Crit_MediumKJ1J2_Corr_L4_J20.005_folder/backup'#,
                #'Crit_MediumKJ1J2_Corr_L4_J20.008_folder/backup',
                #'Crit_MediumKJ1J2_Corr_L4_J20.01_folder/backup',
                'MediumKJ1J2_Corr_L4_J20.005_folder/backup',
                'MediumKJ1J2_Corr_L12_J20.01_folder/backup',
                'MediumKJ1J2_Corr_L4_J20.0125_folder/backup',
                'MediumKJ1J2_Corr_L4_J20.015_folder/backup',
                'MediumKJ1J2_Corr_L8_J20.02_folder/backup',
                'MediumKJ1J2_Corr_L8_J20.03_folder/backup',
                'MediumKJ1J2_Corr_L8_J20.04_folder/backup',
                'MediumKJ1J2_Corr_L8_J20.05_folder/backup',
                'MediumKJ1J2_Corr_L8_J20.06_folder/backup',
                'MediumKJ1J2_Corr_L4_J20.07_folder/backup',
                'MediumKJ1J2_Corr_L4_J20.08_folder/backup',
                'MediumKJ1J2_Corr_L4_J20.1_folder/backup',
                'MediumKJ1J2_Corr_L4_J20.11_folder/backup',
                'MediumKJ1J2_Corr_L4_J20.1125_folder/backup',
                'MediumKJ1J2_Corr_L4_J20.115_folder/backup',
                'MediumKJ1J2_Corr_L4_J20.12_folder/backup',
                'MediumKJ1J2_Corr_L4_J20.13_folder/backup',
                'MediumKJ1J2_Corr_L4_J20.14_folder/backup',
                'MediumKJ1J2_Corr_L4_J20.15_folder/backup'
               ]
os.makedirs('./' + foldername + results_foldername, exist_ok = True)

n = len(filenamelist)

In [ ]:
[L, numsites, J1, J2, J3, J3st, J4, nb, num_in_bin, temperatures, nt,
 stat_temps, temperatures_plots, hfields, nh, 
 stat_hfields, hfields_plots, listfunctions, sref, ids2walker] = \
af.LoadParameters(foldername, filenamelist)

s_ijl, ijl_s = kdraw.createspinsitetable(L[0])
print("nb ", nb, "num_in_bin ", num_in_bin)
swapst_th, swapsh_th, swapst, swapsh = af.LoadSwaps(foldername, filenamelist, nb, num_in_bin, nh, nt)

kw = {'binning': True, 'plzplot': False, 'plotmin': 0, 'plotmax': 16}
[t_h_MeanE, t_h_MeanEsq, t_h_varMeanE, t_h_varMeanEsq, C, ErrC] = \
 af.LoadEnergy(foldername, filenamelist, numsites,
               nb, stat_temps, temperatures, stat_hfields, listfunctions, **kw)

[t_h_MeanM, t_h_MeanMsq, t_h_varMeanM, t_h_varMeanMsq, Chi, ErrChi] = \
 af.LoadMagnetisation(foldername, filenamelist, numsites,
               nb, stat_temps, temperatures, stat_hfields, listfunctions, **kw)

In [ ]:
###########################################
        ## ALGORITHM ANALYSIS ##
###########################################
tidmin = 0
tidmax = nt
af.SwapsAnalysis(L, n, tidmin, tidmax, temperatures, hfields,
                 foldername, results_foldername, swapst, swapsh)

In [ ]:
###########################################
tidmin = 0
tidmax = [len(stat_temps[i]) for i in range(n)]
temperatures_plots = np.array(temperatures_plots)
### Energy

In [ ]:
###########################################
        ## Energy ANALYSIS ##
###########################################
### Energy
S0 = np.log(2)
kw = {'gscheck': True}
af.BasicPlotsE(L, n, tidmin, tidmax, temperatures_plots, hfields_plots, foldername,
                results_foldername, filenamelist, t_h_MeanE, t_h_MeanEsq, t_h_varMeanE,
                t_h_varMeanEsq, C, ErrC, J1, J2, J3, J4, **kw)
plt.show()

In [ ]:
###########################################
        ## MAGNETISATION ANALYSIS ##
###########################################
af.BasicPlotsM(L, n, tidmin, tidmax, temperatures_plots, hfields_plots, foldername,
                results_foldername, filenamelist, t_h_MeanM, t_h_MeanMsq, t_h_varMeanM,
                t_h_varMeanMsq, Chi, ErrChi, J1, J2, J3, J4, **kw)
plt.show()

In [ ]:
rmmag = True
kw = {'rmmag':rmmag,'binning': True, 'plzplot':False}

In [ ]:
t_h_MeanFc, t_h_varMeanFc, t_h_MeanSi, t_h_varMeanSi= \
 af.LoadFirstCorrelations(foldername, filenamelist, listfunctions,stat_temps,
                          stat_hfields, nb,t_h_varMeanMsq, **kw)

In [ ]:
t_h_MeanSs, t_h_varMeanSs, t_h_MeanSi, t_h_varMeanSi, t_h_MeanCorr, t_h_errCorrEstim = \
 af.LoadCentralCorrelations(foldername, filenamelist, listfunctions, sref, stat_temps, stat_hfields, nb, **kw)
print(t_h_errCorrEstim[0].shape)

In [ ]:
for i in range(n):
    addsave = "J2ovJ1" +"={0}".format(J2[i]/J1[i])
    addtitle = r"$J_2/J_1$={0}".format(J2[i]/J1[i])
    af.BasicPlotsFirstCorrelations(L, i, t_h_MeanFc, temperatures_plots, 
                                   t_h_varMeanFc, foldername, 
                                   results_foldername, filenamelist,
                                   tmin = 0, setyticks = np.arange(-0.3,0.7,0.1),
                                   addsave = addsave,
                                   addtitle = addtitle)

In [ ]:
# Introducing the experimental values for the plots below:
## <sisj>-<si><sj>:
NN1exp = -0.22;
NN1experr = 0.0097;
NN2exp = 0.021;
NN2experr = 0.009;
NN3pexp = 0.065;
NN3pexperr = 0.012;
NN3sexp = -0.001;
NN3sexperr = 0.011;

mexp = -0.193;

rmexpmag = True

if not rmexpmag:
    NN1exp += mexp**2;
    NN2exp += mexp**2;
    NN3pexp += mexp**2;
    NN3sexp += mexp**2;
    

print(NN1exp)
print(NN2exp)
print(NN3pexp)
print(NN3sexp)

In [ ]:
for i in range(n):
    addsave = "J2ovJ1" +"={0}".format(J2[i]/J1[i])
    addtitle = r"$J_2/J_1$={0}".format(J2[i]/J1[i])
    af.BasicPlotsFirstCorrelations(L, i, t_h_MeanFc, temperatures_plots, 
                                   t_h_varMeanFc, foldername, 
                                   results_foldername, filenamelist,
                                   tmin = 0, setyticks = np.arange(-0.3,0.7,0.1),
                                   addtitle = addtitle,
                                   save = False)
    plt.gca().set_prop_cycle(None)
    #plt.plot([1e-3,60],[NN1exp, NN1exp])
    plt.fill_between([1e-3,60],[NN1exp-NN1experr,NN1exp-NN1experr],[NN1exp+NN1experr, NN1exp+NN1experr], alpha = 0.2, label = 'NN1 - exp')
    #plt.plot([1e-3,60],[NN2exp, NN2exp])
    plt.fill_between([1e-3,60],[NN2exp-NN2experr,NN2exp-NN2experr],[NN2exp+NN2experr, NN2exp+NN2experr], alpha = 0.2, label = 'NN2 - exp')
    #plt.plot([1e-3,60],[NN3pexp, NN3pexp])
    plt.fill_between([1e-3,60],[NN3pexp-NN3pexperr,NN3pexp-NN3pexperr],[NN3pexp+NN3pexperr, NN3pexp+NN3pexperr], alpha = 0.2, label = 'NN3par - exp')
    #plt.plot([1e-3,60],[NN3sexp, NN3sexp])
    plt.fill_between([1e-3,60],[NN3sexp-NN3sexperr, NN3sexp-NN3sexperr],[NN3sexp+NN3sexperr, NN3sexp+NN3sexperr], alpha = 0.2, label = 'NN3star - exp')
    plt.xlim([0.1, 20])
    plt.ylim([-0.35, 0.2])
    plt.yticks(np.arange(-0.35,0.25,0.05))
    plt.legend()
    plt.savefig("./" + foldername + results_foldername + "/FirstCorrelations_Zoom" + addsave +"rmexpmag={0}.png".format(rmexpmag))

In [ ]:
#for i in range(n):
#    addsave = "J2ovJ1" +"={0}".format(J2[i]/J1[i])
#    addtitle = r"$J_2/J_1$={0}".format(J2[i]/J1[i])
#    af.BasicPlotsFirstCorrelations(L, i, t_h_MeanFc, temperatures_plots, 
#                                   t_h_varMeanFc, foldername, 
#                                   results_foldername, filenamelist,
#                                   tmin = 0, setyticks = np.arange(-0.3,0.7,0.1),
#                                   addtitle = addtitle,
#                                   save = False, log = False)
#    plt.gca().set_prop_cycle(None)
#    plt.fill_between([1e-3,60],[NN1exp-NN1experr,NN1exp-NN1experr],[NN1exp+NN1experr, NN1exp+NN1experr], alpha = 0.2, label = 'NN1 - exp')
#    plt.fill_between([1e-3,60],[NN2exp-NN2experr,NN2exp-NN2experr],[NN2exp+NN2experr, NN2exp+NN2experr], alpha = 0.2, label = 'NN2 - exp')
#    plt.fill_between([1e-3,60],[NN3pexp-NN3pexperr,NN3pexp-NN3pexperr],[NN3pexp+NN3pexperr, NN3pexp+NN3pexperr], alpha = 0.2, label = 'NN3par - exp')
#    plt.fill_between([1e-3,60],[NN3sexp-NN3sexperr, NN3sexp-NN3sexperr],[NN3sexp+NN3sexperr, NN3sexp+NN3sexperr], alpha = 0.2, label = 'NN3star - exp')
#
#    plt.xlim([0, 8])
#    plt.ylim([-0.35, 0.2])
#    plt.yticks(np.arange(-0.35,0.25,0.05))
#    plt.legend()
#    plt.savefig("./" + foldername + results_foldername + "/FirstCorrelations_Zoom_Linear" + addsave +".png")

In [ ]:
#for i in range(n):
#    addsave = "J2ovJ1" +"={0}".format(J2[i]/J1[i])
#    addtitle = r"$J_2/J_1$={0}".format(J2[i]/J1[i])
#    af.BasicPlotsFirstCorrelations(L, i, t_h_MeanFc, temperatures_plots, 
#                                   t_h_varMeanFc, foldername, 
#                                   results_foldername, filenamelist,
#                                   tmin = 0, setyticks = np.arange(-0.3,0.7,0.1),
#                                   addtitle = addtitle,
#                                   save = False, log = False)
#    plt.gca().set_prop_cycle(None)
#    plt.plot([1e-3,60],[-0.179, -0.179])
#    plt.fill_between([1e-3,60],[-0.2, -0.2], [-0.157, -0.157], alpha = 0.2)
#    plt.plot([1e-3,60],[0.059, 0.059])
#    plt.fill_between([1e-3,60],[0.041, 0.041], [0.077, 0.077], alpha = 0.2)
#    plt.plot([1e-3,60],[0.1, 0.1])
#    plt.fill_between([1e-3,60],[0.076, 0.076], [0.124, 0.124], alpha = 0.2)
#    plt.plot([1e-3,60],[0.03, 0.03])
#    plt.fill_between([1e-3,60],[0.023, 0.023], [0.057, 0.057], alpha = 0.2)
#
#    #plt.xlim([3.5, 5.5])
#    plt.ylim([-0.05,0])
#    #plt.yticks(np.arange(-0.35,0.25,0.05))
#    #plt.xticks(np.arange(3.5,5.5,0.1))
#    plt.legend()
#    plt.savefig("./" + foldername + results_foldername + "/FirstCorrelations_ZoomJ1_" + addsave +".png")
#

In [ ]:
for i in range(n):
    addsave = "J2ovJ1" +"={0}".format(J2[i]/J1[i])
    addtitle = r"$J_2/J_1$={0}".format(J2[i]/J1[i])
    af.BasicPlotsDifferenceFirstCorrelations(L, i, t_h_MeanFc, temperatures_plots, t_h_varMeanFc,
                                    foldername, results_foldername, filenamelist, 
                                    tmin = 0, tmax = 128, setxlim = None, setylim = None, 
                                    setxticks = None, setyticks = None, 
                                    addtitle = addtitle, addsave = addsave,
                                    save = True, log = True)

In [ ]:
for i in range(3):
    addsave = "J2ovJ1" +"={0}".format(J2[i]/J1[i])
    addtitle = r"$J_2/J_1$={0}".format(J2[i]/J1[i])
    af.BasicPlotsDifferenceFirstCorrelations(L, i, t_h_MeanFc, temperatures_plots, t_h_varMeanFc,
                                    foldername, results_foldername, filenamelist, 
                                    tmin = 0, tmax = 128, setxlim = [1, 60], setylim = [-0.02, 0.05], 
                                    setxticks = None, setyticks =None, 
                                    addtitle = addtitle, addsave = addsave,
                                    save = False, log = False)
    plt.savefig("./" + foldername + results_foldername + "/FirstCorrelationsDifference"+addsave+"_ZoomLinear.png")

In [ ]:
#for i in range(n):
#    addsave = "J2ovJ1" +"={0}".format(J2[i]/J1[i])
#    addtitle = r"$J_2/J_1$={0}".format(J2[i]/J1[i])
#    af.BasicPlotsDifferenceFirstCorrelations(L, i, t_h_MeanFc, temperatures_plots, t_h_varMeanFc,
#                                    foldername, results_foldername, filenamelist, 
#                                    tmin = 0, tmax = 128, setxlim = [0.01, 60], setylim = [-0.2, 0.05], 
#                                    setxticks = None, setyticks =np.arange(-0.2,0.075,0.025), 
#                                    addtitle = addtitle, addsave = addsave,
#                                    save = False, log = True)
#    plt.savefig("./" + foldername + results_foldername + "/FirstCorrelationsDifference"+addsave+"_ZoomLog.png")

In [ ]:
#for i in range(n):
#    addsave = "J2ovJ1" +"={0}".format(J2[i]/J1[i])
#    addtitle = r"$J_2/J_1$={0}".format(J2[i]/J1[i])
#    af.BasicPlotsDifferenceFirstCorrelations(L, i, t_h_MeanFc, temperatures_plots, t_h_varMeanFc,
#                                    foldername, results_foldername, filenamelist, 
#                                    tmin = 0, tmax = 128, setxlim = [4, 6], setylim = [-0.02, 0.005], 
#                                    setxticks = None, setyticks =None, 
#                                    addtitle = addtitle, addsave = addsave,
#                                    save = False, log = False)
#    plt.savefig("./" + foldername + results_foldername + "/FirstCorrelationsDifference"+addsave+"_Measure.png")

In [ ]:
#for i in range(n):
#    addsave = "J2ovJ1" +"={0}".format(J2[i]/J1[i])
#    addtitle = r"$J_2/J_1$={0}".format(J2[i]/J1[i])
#    af.BasicPlotsDifferenceFirstCorrelations(L, i, t_h_MeanFc, temperatures_plots, t_h_varMeanFc,
#                                    foldername, results_foldername, filenamelist, 
#                                    tmin = 0, tmax = 128, setxlim = [2,20], setylim = [-0.05, 0.05], 
#                                    setxticks = None, setyticks =None, 
#                                    addtitle = addtitle, addsave = addsave,
#                                    save = False, log = False)
#    plt.savefig("./" + foldername + results_foldername + "/FirstCorrelationsDifference"+addsave+"_Measure2.png")

In [ ]:
ploth = False
tmin = 0
tmax = 128

In [ ]:
#### COMPUTING CORRELATIONS FROM <SISJ> AND <SI> and plotting in 2D
#
#if not ploth:
#    tid = 0
#    hid = 0
#    rid = tid
#else:
#    tid = 0
#    hid = 0
#    rid = hid
#
#[corr, errcorr, maxerr] = af.PrepPlot2DCorrelations(rid, n, 
#                                                t_h_MeanCorr,
#                                                t_h_errCorrEstim,
#                                                t_h_MeanSi,
#                                                hfields_plots,
#                                                temperatures_plots,\
#                                                ploth = ploth)
#print("t = ",temperatures_plots[0][tid])
#print("h = ",hfields_plots[0][hid])

In [ ]:
for i in range(n):
    addsave = "J2ovJ1" +"={0}".format(J2[i]/J1[i])
    addtitle = r" $J_2/J_1$={0}".format(J2[i]/J1[i])
    af.PlotFirstCorrelations(i, L,foldername, results_foldername,  hfields_plots, temperatures_plots,
                             t_h_MeanCorr, t_h_errCorrEstim, sref, \
                             distmax = 2, ploth = ploth, plotFirst = True,
                            t_h_MeanFc = t_h_MeanFc, tmin = tmin, tmax = tmax, 
                             addtitle = addtitle, addsave = addsave, **kw)

In [ ]:
#### Values at Exp. T ####

In [ ]:
Tmin = 2
Tmax = 6

In [ ]:
idTmin = []
idTmax = []

for i in range(n):
    idTmin.append(np.abs(np.array(temperatures_plots[i])-Tmin).argmin())
    idTmax.append(np.abs(np.array(temperatures_plots[i])-Tmax).argmin())

print(idTmin)
print(idTmax)

In [ ]:
NN1 = []
ErrNN1 = []
NN2 = []
ErrNN2 = []
NN3par = []
ErrNN3par = []
NN3star = []
ErrNN3star = []
for i in range(n):
    #for idT in range(idTmin, idTmax):
    NN1.append((t_h_MeanFc[i][idTmin[i]:idTmax[i],0,0]+t_h_MeanFc[i][idTmin[i]+1:idTmax[i]+1,0,0])/2)
    ErrNN1.append(np.sqrt(t_h_varMeanFc[i][idTmin[i]:idTmax[i],0,0]))
    NN2.append((t_h_MeanFc[i][idTmin[i]:idTmax[i],0,1]+t_h_MeanFc[i][idTmin[i]+1:idTmax[i]+1,0,1])/2)
    ErrNN2.append(np.sqrt(t_h_varMeanFc[i][idTmin[i]:idTmax[i],0,1]))
    NN3par.append((t_h_MeanFc[i][idTmin[i]:idTmax[i],0,2]+t_h_MeanFc[i][idTmin[i]+1:idTmax[i]+1,0,2])/2)
    ErrNN3par.append(np.sqrt(t_h_varMeanFc[i][idTmin[i]:idTmax[i],0,2]))
    NN3star.append((t_h_MeanFc[i][idTmin[i]:idTmax[i],0,3]+t_h_MeanFc[i][idTmin[i]+1:idTmax[i]+1,0,3])/2)
    ErrNN3star.append(np.sqrt(t_h_varMeanFc[i][idTmin[i]:idTmax[i],0,3]))

NN1 = np.array(NN1).transpose()
ErrNN1 = np.array(ErrNN1).transpose()

NN2 = np.array(NN2).transpose()
ErrNN2 = np.array(ErrNN2).transpose()

NN3par = np.array(NN3par).transpose()
ErrNN3par = np.array(ErrNN3par).transpose()

NN3star = np.array(NN3star).transpose()
ErrNN3star = np.array(ErrNN3star).transpose()

In [ ]:
print(NN1)

In [ ]:
for idT in range(idTmax[0]-idTmin[0]):
    plt.figure(figsize=(11,9), dpi = 200)
    plt.errorbar(J2, NN1[idT,:],fmt='.', yerr = ErrNN1[idT,:], label = "NN1")
    plt.errorbar(J2, NN2[idT,:],fmt='x',yerr = ErrNN2[idT,:], label = "NN2")
    plt.errorbar(J2, NN3par[idT,:],fmt='v', yerr = ErrNN3par[idT,:], label = "NN3par")
    plt.errorbar(J2, NN3star[idT,:],fmt='*',yerr = ErrNN3star[idT,:], label = "NN3star")
    plt.xlabel(r'$J_2/J_1$')
    plt.ylabel(r'$\langle \sigma_i \sigma_j \rangle - \langle \sigma_i \rangle \langle \sigma_j \rangle$')
    plt.title(r'$T/J_1$ = {:.2f}'.format(temperatures_plots[0][idTmin[0]+idT]))
    plt.ylim([-0.275, 0.15])
    plt.xlim([0,0.15])
    plt.yticks(np.arange(-0.275, 0.1525, 0.025))
    plt.grid(which = 'both')
    plt.legend(loc='best')
    plt.savefig("./" + foldername + results_foldername + "/FirstCorrelations_T={:.2f}.png".format(temperatures_plots[0][idTmin[0]+idT]))

In [ ]:
for idT in range(idTmax[0]-idTmin[0]):
    plt.figure(figsize=(11,9), dpi = 200)
    plt.errorbar(J2, NN1[idT,:],fmt='.', yerr = ErrNN1[idT,:], label = "NN1")
    plt.errorbar(J2, NN2[idT,:],fmt='x',yerr = ErrNN2[idT,:], label = "NN2")
    plt.errorbar(J2, NN3par[idT,:],fmt='v', yerr = ErrNN3par[idT,:], label = "NN3par")
    plt.errorbar(J2, NN3star[idT,:],fmt='*',yerr = ErrNN3star[idT,:], label = "NN3star")
    plt.gca().set_prop_cycle(None)
    plt.fill_between([0,0.15],[NN1exp-NN1experr,NN1exp-NN1experr],[NN1exp+NN1experr, NN1exp+NN1experr], alpha = 0.2, label = 'NN1 - exp')
    plt.fill_between([0,0.15],[NN2exp-NN2experr,NN2exp-NN2experr],[NN2exp+NN2experr, NN2exp+NN2experr], alpha = 0.2, label = 'NN2 - exp')
    plt.fill_between([0,0.15],[NN3pexp-NN3pexperr,NN3pexp-NN3pexperr],[NN3pexp+NN3pexperr, NN3pexp+NN3pexperr], alpha = 0.2, label = 'NN3par - exp')
    plt.fill_between([0,0.15],[NN3sexp-NN3sexperr, NN3sexp-NN3sexperr],[NN3sexp+NN3sexperr, NN3sexp+NN3sexperr], alpha = 0.2, label = 'NN3star - exp')
    plt.xlabel(r'$J_2/J_1$')
    plt.ylabel(r'$\langle \sigma_i \sigma_j \rangle - \langle \sigma_i \rangle \langle \sigma_j \rangle$')
    plt.title(r'$T/J_1$ = {:.2f}'.format(temperatures_plots[0][idTmin[0]+idT]))
    plt.ylim([-0.275, 0.15])
    plt.xlim([0,0.15])
    plt.yticks(np.arange(-0.275, 0.1525, 0.025))
    plt.grid(which = 'both')
    plt.legend(loc='upper center', ncol = 4)
    plt.savefig("./" + foldername + results_foldername + "/FirstCorrelations_T={:.2f}".format(temperatures_plots[0][idTmin[0]+idT])+"_rmexpmag={0}.png".format(rmexpmag))